In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ln -s /content/gdrive/My\ Drive/MathWorks\ \#2\ \(BOS\)\ -\ Classify\ Object\ Behavior\ to\ Enhance\ the\ Safety\ of\ Autonomous\ Vehicles/ /myfiles
!ls /myfiles

Mounted at /content/gdrive
'Bridge To Studio'
 Datasets_Samples
'Fall 2023 AI Studio | Final Project Presentation.gslides'
'MathWorks #2 (BOS) - Classify Object Behavior to Enhance the Safety of Autonomous Vehicles'
'Meeting Notes'
 models
 ModelTrainTest
'[Overview Slide] BOS_MathWorks_Classify Object Behavior to Enhance the Safety of Autonomous Vehicles.gslides'
'Presentation Notes.gdoc'
'Project Scope and Deliverables_Fall 2023 AI Studio.gdoc'
 Resources
 SlidesMeetings
 Vanessa_ModelTraining


In [2]:
BASEPATH = '/myfiles/ModelTrainTest/'
DATA_BASEPATH = BASEPATH+'data/'


labelmap_path = DATA_BASEPATH+'label_map.pbtxt'
output_directory = BASEPATH+'inference'

In [3]:
import os
import pathlib
import glob
import cv2 as cv2
import numpy as np
import xml.etree.ElementTree as ET
import pandas as pd


In [4]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4065, done.
remote: Counting objects: 100% (4065/4065), done.
remote: Compressing objects: 100% (3089/3089), done.
remote: Total 4065 (delta 1186), reused 1956 (delta 916), pack-reused 0
Receiving objects: 100% (4065/4065), 54.71 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


In [5]:
#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
     ━━━

In [6]:
#Loading the saved_model
import tensorflow as tf
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

IMAGE_SIZE = (12, 8) # Output display size as you want
import matplotlib.pyplot as plt
PATH_TO_SAVED_MODEL=  output_directory+'/saved_model'
print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

Loading model...Done!


In [12]:
#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap(labelmap_path,use_display_name=True)
#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))

#Pick an image from the testing set
#Several examples below:
#image_path = DATA_BASEPATH+"images/n003-2018-01-08-12-04-22+0800__CAM_FRONT__1515384776405299.jpg"  #great
#image_path = DATA_BASEPATH+"images/n008-2018-05-30-15-20-59-0400__CAM_FRONT__1527708442112465.jpg"
#image_path = DATA_BASEPATH+"images/n008-2018-09-14-13-52-24-0400__CAM_FRONT__1536947850912404.jpg"
#image_path = DATA_BASEPATH+"images/n009-2018-09-12-09-36-56-0400__CAM_FRONT__1536759538162656.jpg"   #great
#image_path = DATA_BASEPATH+"images/n010-2018-08-03-16-43-05+0800__CAM_FRONT_RIGHT__1533286261620905.jpg" #ok
#image_path = DATA_BASEPATH+"images/n008-2018-09-14-13-25-13-0400__CAM_FRONT_LEFT__1536946292404799.jpg" #not-great
#image_path = DATA_BASEPATH+"images/n008-2018-09-13-15-21-54-0400__CAM_FRONT_LEFT__1536867179254799.jpg"  #just detects 1!
image_path = DATA_BASEPATH+"images/n008-2018-09-10-17-01-47-0400__CAM_FRONT_LEFT__1536613777604799.jpg"

#print('Running inference for {}... '.format(image_path), end='')

image_np = load_image_into_numpy_array(image_path)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.4, # Adjust this value to set the minimum probability boxes to be classified as True
      agnostic_mode=False)
%matplotlib inline
plt.figure(figsize=IMAGE_SIZE, dpi=200)
plt.axis("off")
plt.imshow(image_np_with_detections)
plt.show()

Output hidden; open in https://colab.research.google.com to view.